In [85]:
# pip install openpyxl ffmpeg


In [86]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from IPython.display import HTML
from scipy.interpolate import griddata
from matplotlib.animation import FuncAnimation, PillowWriter

In [87]:
data = pd.read_excel("Sensor data.xlsx",sheet_name=None)

In [88]:
data

{'0474D T|RH IP67(0474D-S1)':                 Sensor ID             0474D    Unnamed: 2
 0             Sensor Name   0474D T/RH IP67           NaN
 1            Sensor Group               NaN           NaN
 2                    Time  Temperature (°C)  Humidity (%)
 3     2024-04-16 13:09:53             21.95          50.3
 4     2024-04-16 13:14:52              22.5          34.7
 ...                   ...               ...           ...
 8358  2024-05-15 13:02:14              29.7            43
 8359  2024-05-15 13:07:13             29.55          44.9
 8360  2024-05-15 13:12:13              30.4          42.7
 8361  2024-05-15 13:17:13                31          44.2
 8362  2024-05-15 13:22:12              31.2          42.1
 
 [8363 rows x 3 columns],
 '04750 T|RH IP67(04750-S1)':                  Sensor ID             04750    Unnamed: 2
 0              Sensor Name   04750 T/RH IP67           NaN
 1             Sensor Group               NaN           NaN
 2                     Tim

In [89]:
def extract_data_from_sheet(df):
    sensor_name = df.iloc[0, 1]

    extracted_df = df.iloc[3:].reset_index(drop=True)

    extracted_df.columns = ['Time', 'Temperature', 'Humidity']
    extracted_df['Time'] = pd.to_datetime(extracted_df['Time'], format='%d-%m-%Y %H:%M')

    extracted_df['Sensor Info'] = sensor_name
    
    # Extracting the first 5 characters from 'Sensor Info' and creating 'Sensor ID' column
    extracted_df['Sensor ID'] = extracted_df['Sensor Info'].str[:5]

    return extracted_df

In [90]:
data_frames = []

for sheet_name, df in data.items():
    extracted_df = extract_data_from_sheet(df)
    data_frames.append(extracted_df)

In [91]:
combined_data = pd.concat(data_frames, ignore_index=True)

In [92]:
combined_data.head()

,Time,Temperature,Humidity,Sensor Info,Sensor ID
0,2024-04-16 13:09:53,21.95,50.3,0474D T/RH IP67,0474D
1,2024-04-16 13:14:52,22.5,34.7,0474D T/RH IP67,0474D
2,2024-04-16 13:19:54,24.05,34.7,0474D T/RH IP67,0474D
3,2024-04-16 13:24:53,24.25,34.7,0474D T/RH IP67,0474D
4,2024-04-16 13:29:52,24.15,35.2,0474D T/RH IP67,0474D


In [93]:
combined_data.isnull().sum()

Time              0
Temperature    2235
Humidity       2235
Sensor Info       0
Sensor ID         0
dtype: int64

In [94]:
# Check for null values
null_counts = combined_data.groupby('Sensor ID')[['Temperature', 'Humidity']].apply(lambda x: x.isnull().sum())

# Filter sensors with null values
sensors_with_nulls = null_counts[(null_counts['Temperature'] > 0) | (null_counts['Humidity'] > 0)]

print("Sensors with null values:")
print(sensors_with_nulls)

Sensors with null values:
           Temperature  Humidity
Sensor ID                       
0474D               25        25
04750               62        62
04754              313       313
04756              300       300
04757               20        20
0475A               23        23
0475C               51        51
0475D               52        52
04762               33        33
04763               54        54
04765              297       297
04768               54        54
0476E               77        77
04771              113       113
04773              290       290
0477F               40        40
04787              313       313
04789               60        60
0478A               26        26
0478C               32        32


In [95]:
len(combined_data['Sensor Info'].unique())

20

In [96]:
sample_dataset = combined_data.copy()

In [97]:
sample_dataset = sample_dataset[['Time','Humidity','Sensor ID']]
sample_dataset.head()

,Time,Humidity,Sensor ID
0,2024-04-16 13:09:53,50.3,0474D
1,2024-04-16 13:14:52,34.7,0474D
2,2024-04-16 13:19:54,34.7,0474D
3,2024-04-16 13:24:53,34.7,0474D
4,2024-04-16 13:29:52,35.2,0474D


In [98]:
sample_dataset['Humidity'] = sample_dataset['Humidity'].astype('float')
# Get unique sensor IDs
sensor_ids = sample_dataset['Sensor ID'].unique()

# Define grid size
size_x = 10 # Reduce grid size for demonstration
size_y = 10

# Create a directory to save the plots if it doesn't exist
output_dir = "Sensor_Heatmaps_Imgs"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Plotting heat maps for each sensor
frames = []
for sensor_id in sensor_ids:
    # Filter data for the current sensor
    sensor_data = sample_dataset[sample_dataset['Sensor ID'] == sensor_id].reset_index(drop=True)

    # Initialize Humidity grid
    Humidity_grid = np.zeros((size_x, size_y))

    # Map Humidity readings to the grid
    for index, row in sensor_data.iterrows():
        x = index % size_x
        y = index // size_x
        if x < size_x and y < size_y:
            Humidity_grid[x, y] = row['Humidity']


    # Plot Humidity distribution using heatmap
    plt.figure(figsize=(10, 6))  # Adjust figure size
    ax = sns.heatmap(Humidity_grid, cmap='magma', cbar_kws={'label': 'Humidity (%)'}, fmt=".2f", xticklabels=False, yticklabels=False)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Adjust plot margins

    # Add labels and title
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    ax.set_title(f'Sensor {sensor_id} Humidity Heatmap')

    # Save the plot as an image file
    output_file = os.path.join(output_dir, f"heatmap_{sensor_id}.png")
    plt.savefig(output_file)
    plt.close()

    frames.append(output_file)

print(f"Heat maps saved in the directory: {output_dir}")

Heat maps saved in the directory: Sensor_Heatmaps_Imgs


In [99]:
# Define the output video filename and codec
video_output = "Sensor_Heatmaps_Video.mp4"
frame = cv2.imread(frames[0])
height, width, _ = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use MP4V codec for MP4 format
video = cv2.VideoWriter(video_output, fourcc, 5.0, (width, height))  # Lower frame rate for smoother transition

# Read the first frame
prev_frame = cv2.imread(frames[0])

for frame_path in frames[1:]:
    # Read the next frame
    next_frame = cv2.imread(frame_path)
    
    # Apply optical flow to get intermediate frames
    for i in range(1, 10):  # Generate 10 intermediate frames
        alpha = i / 10.0
        interpolated_frame = cv2.addWeighted(prev_frame, 1 - alpha, next_frame, alpha, 0)
        
        # Write the interpolated frame to the video
        video.write(interpolated_frame)

    prev_frame = next_frame

# Release the video writer
video.release()
cv2.destroyAllWindows()

print(f"Video saved as: {video_output}")

Video saved as: Sensor_Heatmaps_Video.mp4


In [100]:
# # Load and prepare data (assuming 'sample_dataset' is already prepared)
# sample_dataset['Humidity'] = sample_dataset['Humidity'].astype('float')
# sensor_ids = sample_dataset['Sensor ID'].unique()

# # Create a directory to save the plots if it doesn't exist
# output_dir = "sensor_heatmaps"
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # Define a function to create smooth heatmaps
# def create_heatmap(data, sensor_id, output_dir):
#     # Create a grid for interpolation
#     grid_x, grid_y = np.mgrid[0:1:100j, 0:1:100j]

#     # Normalize time for better interpolation
#     time_normalized = (data['Time'] - data['Time'].min()) / (data['Time'].max() - data['Time'].min())

#     # Interpolate humidity data
#     points = np.column_stack((time_normalized, np.zeros(len(data))))
#     values = data['Humidity'].values
#     try:
#         grid_z = griddata(points, values, (grid_x, grid_y), method='nearest')  # Changed to 'nearest' interpolation
#     except Exception as e:
#         print(f"Error interpolating for sensor {sensor_id}: {e}")
#         return None

#     # Plot the heatmap
#     plt.figure(figsize=(10, 6))
#     sns.heatmap(grid_z, cmap='hot', cbar_kws={'label': 'Humidity (°C)'}, fmt=".2f", xticklabels=False, yticklabels=False, cbar=False)
#     plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    
#     output_file = os.path.join(output_dir, f"heatmap_{sensor_id}.png")
#     plt.savefig(output_file)
#     plt.close()
    
#     return output_file

# # Generate heatmaps for each sensor and store frame paths
# frames = []
# for sensor_id in sensor_ids:
#     sensor_data = sample_dataset[sample_dataset['Sensor ID'] == sensor_id].reset_index(drop=True)
#     frame_path = create_heatmap(sensor_data, sensor_id, output_dir)
#     if frame_path:
#         frames.append(frame_path)

# print(f"Heat maps saved in the directory: {output_dir}")

In [101]:
# # Define the output video filename and codec
# video_output = "sensor_heatmaps_video.mp4"
# frame = cv2.imread(frames[0])
# height, width, _ = frame.shape

# fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use MP4V codec for MP4 format
# video = cv2.VideoWriter(video_output, fourcc, 5.0, (width, height))  # Lower frame rate for smoother transition

# # Read the first frame
# prev_frame = cv2.imread(frames[0])

# for frame_path in frames[1:]:
#     # Read the next frame
#     next_frame = cv2.imread(frame_path)
    
#     # Apply optical flow to get intermediate frames
#     for i in range(1, 10):  # Generate 10 intermediate frames
#         alpha = i / 10.0
#         interpolated_frame = cv2.addWeighted(prev_frame, 1 - alpha, next_frame, alpha, 0)
        
#         # Write the interpolated frame to the video
#         video.write(interpolated_frame)

#     prev_frame = next_frame

# # Release the video writer
# video.release()
# cv2.destroyAllWindows()

# print(f"Video saved as: {video_output}")